Based on https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

  Using cached tensorflow-2.9.1-cp310-cp310-win_amd64.whl (444.1 MB)
     -------------------------------------- 444.1/444.1 MB 3.1 MB/s eta 0:00:00
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
     ---------------------------------------- 7.2/7.2 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00
     -------------------------------------- 895.5/895.5 kB 4.0 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ---------------------------------------- 2.6/2.6 MB 4.5 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using

In [2]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:
# Set GPU growth to avoid out of memory errors

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [4]:
# Setting paths
POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data','negative')
ANC_PATH = os.path.join('data','anchor')

# # Making the directories
# os.makedirs(POS_PATH)
# os.makedirs(NEG_PATH)
# os.makedirs(ANC_PATH)

Labeled data from http://vis-www.cs.umass.edu/lfw/

In [5]:
# Uncompress the data and move it to negative folder
# !tar -xf lfw.tgz
# for directory in os.listdir('lfw'):
#   for file_name in os.listdir(os.path.join('lfw', directory)):
#     EX_PATH = os.path.join('lfw', directory, file_name)
#     NEW_PATH = os.path.join(NEG_PATH, file_name)
#     os.replace(EX_PATH, NEW_PATH)


In [15]:
# Import uuid library to generate unique image names
import uuid

# Establish a connection to the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,150:150+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [14]:
# Preprocess images

anchor = tf.data.Dataset.list_files(ANC_PATH+ '\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+ '\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+ '\*.jpg').take(300)

def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100))
    img = img/255.0
    return img

positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)
    
# Data pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)


In [19]:
sample_size = round(len(data)* 0.7)
train_data = data.take(sample_size)
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

test_data = data.skip(sample_size)
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)